In [42]:
%load_ext blackcellmagic
%load_ext autoreload
%autoreload 2

The blackcellmagic extension is already loaded. To reload it, use:
  %reload_ext blackcellmagic
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# !python scripts/mic_2_generate_animals_index.py
# !python scripts/mic_3_generate_trajectories_index.py
# !python scripts/mic_4_generate_master_videos_table.py 
# !python scripts/mic_5_plot_tracking_sanity_check.py
# !python scripts/mic_6_generate_tr_datasets.py
# !python scripts/mic_7_plot_summary_figures.py

In [45]:
import os

from ipywidgets import interact
from confapp import conf
import pandas as pd
import numpy as np

from PIL import Image
from IPython.display import display
from natsort import natsorted

from mic_analysis.plot_interactive import display_partition_figures, get_possible_variables, get_possible_outliers, visualize_outlier

# Visualize pre-plotted figures

In [46]:
videos_table = pd.read_csv(conf.VIDEOS_INDEX_FILE_NAME)

partition_col = "line_experiment"
video_uid_col='trial_uid'

possible_partition_uids = natsorted(
    videos_table[partition_col].unique()
)

save_path = os.path.join(
    conf.GENERATED_FIGURES_PATH, f"summary_{partition_col}"
)
if not os.path.isdir(save_path):
    os.makedirs(save_path)
    
    
@interact(partition_uid=possible_partition_uids, var_type=['indiv', 'group', 'indiv_nb'], visualize_outliers=False)
def summary_partition(partition_uid='ap1g1_1_1', var_type='indiv', visualize_outliers=False):
    
    load_path = os.path.join(save_path, partition_uid)
    display_partition_figures(load_path, var_type)
        
    if visualize_outliers:   
        outliers_path = os.path.join(load_path, "all_outliers.csv")
        if os.path.isfile(outliers_path):
            outliers = pd.read_csv(outliers_path)
            outliers_fig_path = os.path.join(load_path, "outliers_summaries")
            variables = get_possible_variables(outliers)
                        
            @interact(variable=variables)
            def show_variable(variable):         
                outliers_uids = get_possible_outliers(outliers, variable, video_uid_col)
                
                @interact(outlier=outliers_uids)
                def plot_outlier_var(outlier):
                    visualize_outlier(outliers, outlier, outliers_fig_path, variable, video_uid_col)
        

interactive(children=(Dropdown(description='partition_uid', options=('ap1g1_1_1', 'apbb1_1_1', 'asap1b_7_1', '…